In [1]:
import json
import os

from tqdm import tqdm
from xmltodict import unparse

In [2]:
src_base = "../yolo/COCO/annotations"
dst_base =  "../yolo/COCO/VOCtmp"
# may change to 1
BBOX_OFFSET = 0

In [3]:
dst_dirs = {x: os.path.join(dst_base, x) for x in ["Annotations", "ImageSets", "JPEGImages"]}

In [4]:
dst_dirs['ImageSets'] = os.path.join(dst_dirs['ImageSets'], "Main")

In [5]:
for k, d in dst_dirs.items():
    os.makedirs(d, exist_ok=True)

In [6]:
def base_dict(filename, width, height, depth=3):
    return {
        "annotation": {
            "filename": os.path.split(filename)[-1],
            "folder": "VOCtmp", "segmented": "0", "owner": {"name": "unknown"},
            "source": {'database': "The COCO 2017 database", 'annotation': "COCO 2017", "image": "unknown"},
            "size": {'width': width, 'height': height, "depth": depth},
            "object": []
        }
    }

In [7]:
def base_object(size_info, name, bbox):
    x1, y1, w, h = bbox
    x2, y2 = x1 + w, y1 + h

    width = size_info['width']
    height = size_info['height']

    x1 = int(max(x1, 0) + BBOX_OFFSET)
    y1 = int(max(y1, 0) + BBOX_OFFSET)
    x2 = int(min(x2, width - 1) + BBOX_OFFSET)
    y2 = int(min(y2, height - 1) + BBOX_OFFSET)

    return {
        'name': name, 'pose': 'Unspecified', 'truncated': '0', 'difficult': '0',
        'bndbox': {'xmin': x1, 'ymin': y1, 'xmax': x2, 'ymax': y2}
    }

In [8]:
sets = {
    # for test only!
    "val": os.path.join(src_base, "instances_val2017.json")
}

In [9]:
cate = {x['id']: x['name'] for x in json.load(open(sets["val"]))['categories']}

In [10]:
for stage, filename in sets.items():
    print("Parse", filename)
    data = json.load(open(filename))

Parse ../yolo/COCO/annotations\instances_val2017.json


In [11]:
    images = {}
    for im in tqdm(data["images"], "Parse Images"):
        img = base_dict(im['coco_url'], im['width'], im['height'], 3)
        images[im["id"]] = img

Parse Images: 100%|████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 108987.69it/s]


In [12]:
    for an in tqdm(data["annotations"], "Parse Annotations"):
        ann = base_object(images[an['image_id']]['annotation']["size"], cate[an['category_id']], an['bbox'])
        images[an['image_id']]['annotation']['object'].append(ann)

Parse Annotations: 100%|█████████████████████████████████████████████████████| 36781/36781 [00:00<00:00, 162635.87it/s]


In [20]:
    del_keys = []
    for image in images:
        if images[image]['annotation']['object'] == []:
            print(image)
            del_keys.append(image)
    for key in del_keys:
        del images[key]

226111
58636
458790
461275
268996
370999
481404
121153
310622
98497
228771
270386
240767
173183
560371
330554
550939
502910
382734
344611
127135
320706
64574
198915
101022
556498
308391
25593
536343
477118
528977
42888
41488
514540
374727
402096
200152
542073
476491
278006
261796
312549
260657
267946
49091
176701
404601
447789


In [21]:
    images[226111]

KeyError: 226111

In [22]:
    for k, im in tqdm(images.items(), "Write Annotations"):
        im['annotation']['object'] = im['annotation']['object'] or [None]
        unparse(im,
                open(os.path.join(dst_dirs["Annotations"], "{}.xml".format(str(k).zfill(12))), "w"),
                full_document=False, pretty=True)

Write Annotations: 100%|██████████████████████████████████████████████████████████| 4952/4952 [00:40<00:00, 123.74it/s]


In [23]:
    print("Write image sets")
    with open(os.path.join(dst_dirs["ImageSets"], "{}.txt".format(stage)), "w") as f:
        f.writelines(list(map(lambda x: str(x).zfill(12) + "\n", images.keys())))

Write image sets


In [24]:
import os
import sys
import xml.etree.ElementTree as ET
sys.path.insert(0, '../yolo/')
import brambox.boxes as bbb

In [25]:
DEBUG = True        # Enable some debug prints with extra information
ROOT = 'C:\\Users\\AndrewHuang\\Documents\\GitHub\\ObjectDetection-OneStageDet\\yolo\\COCO'       # Root folder where the VOCdevkit is located

In [26]:
def identify(xml_file):
    root_dir = ROOT
    root = ET.parse(xml_file).getroot()
    folder = root.find('folder').text
    filename = root.find('filename').text
    print(f'{root_dir}/{folder}/JPEGImages/{filename}')
    return f'{root_dir}/{folder}/JPEGImages/{filename}'

In [27]:
if __name__ == '__main__':
    print('Getting training annotation filenames')
    train = []
    with open(f'{ROOT}/VOCtmp/ImageSets/Main/val.txt', 'r') as f:
        ids = f.read().strip().split()
    train += [f'{ROOT}/VOCtmp/Annotations/{xml_id}.xml' for xml_id in ids]

    if DEBUG:
        print(f'\t{len(train)} xml files')

Getting training annotation filenames
	4952 xml files


In [28]:
    print('Parsing training annotation files')
    train_annos = bbb.parse('anno_pascalvoc', train, identify)
    # Remove difficult for training
    for k,annos in train_annos.items():
        for i in range(len(annos)-1, -1, -1):
            if annos[i].difficult:
                del annos[i]

Parsing training annotation files
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000397133.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000037777.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000252219.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000087038.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000174482.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000403385.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000006818.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000480985.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VO

C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000109798.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000370677.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000238866.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000369370.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000502737.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000515579.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000515445.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000173383.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000438862.jpg
C

C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000278848.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000166391.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000048153.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000459153.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000295713.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000223130.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000273132.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000198960.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000344059.jpg
C

C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000084431.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000323355.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000355325.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000100582.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000555412.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000004495.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000009483.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000326082.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000398237.jpg
C

C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000337055.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000110638.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000034139.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000080340.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000083113.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000173033.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000255664.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000072813.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000545129.jpg
C

...

C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000370900.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000528705.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000050844.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000174018.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000009590.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000018380.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000016249.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000281032.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000199771.jpg
C

C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000532071.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000054123.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000299609.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000080659.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000303653.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000451144.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000386210.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000085682.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000465806.jpg
C

C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000466156.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000111086.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000564023.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000069213.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000292456.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000453634.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000155154.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000013177.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000037751.jpg
C

C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000054628.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000157928.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000203095.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000017182.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000150649.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000562059.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000407524.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000306437.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000116206.jpg
C

C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000507042.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000314541.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000341921.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000186449.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000547336.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000226154.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000394275.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000229753.jpg
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/VOCtmp/JPEGImages/000000261712.jpg
C

In [29]:
train_annos

{'C:\\Users\\AndrewHuang\\Documents\\GitHub\\ObjectDetection-OneStageDet\\yolo\\COCO/VOCtmp/JPEGImages/000000397133.jpg': [PascalVocAnnotation {class_label = 'bottle', object_id = 0, x = 217.0, y = 240.0, w = 40.0, h = 59.0, ignore = False, lost = None, difficult = False, truncated_fraction = 0.0, occluded_fraction = 0.0, visible_x = 0.0, visible_y = 0.0, visible_w = 0.0, visible_h = 0.0},
  PascalVocAnnotation {class_label = 'dining table', object_id = 0, x = 1.0, y = 240.0, w = 347.0, h = 187.0, ignore = False, lost = None, difficult = False, truncated_fraction = 0.0, occluded_fraction = 0.0, visible_x = 0.0, visible_y = 0.0, visible_w = 0.0, visible_h = 0.0},
  PascalVocAnnotation {class_label = 'person', object_id = 0, x = 388.0, y = 69.0, w = 111.0, h = 279.0, ignore = False, lost = None, difficult = False, truncated_fraction = 0.0, occluded_fraction = 0.0, visible_x = 0.0, visible_y = 0.0, visible_w = 0.0, visible_h = 0.0},
  PascalVocAnnotation {class_label = 'knife', object_id 

In [31]:
bbb.generate('anno_pickle', train_annos, f'{ROOT}/onedet_cache/val_coco.pkl')